<a href="https://colab.research.google.com/github/LittlePandaCode/CodeAlpha_Credit_Scoring_Mode/blob/main/codeAlpha_Emotion_Recognition_from__Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraraies

In [ ]:
!pip install librosa
!pip install tensorflow
from tensorflow import keras
!pip install librosa soundfile numpy sklearn pyaudio
!pip install soundfile
!pip install resampy

## Make the necessary imports

In [12]:
import os
import librosa
import numpy as np
import resampy
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tensorflow.keras import models, layers, optimizers, callbacks

In [13]:
audio_path= r'/content/drive/MyDrive/Datasets/audio_speech_actors'

In [14]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}
observed_emotions=['neutral','calm','happy','sad','angry','fearful','disgust','surprised']

## Load Dataset and Extract Features

In [15]:
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    audio, sr = librosa.load(file_path, sr=None)
    features = []
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40).T, axis=0)
        features.extend(mfccs)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr).T,axis=0)
        features.extend(chroma)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr).T,axis=0)
        features.extend(mel)
    return features

# Load and preprocess data
data = []
labels = []
for actor_folder in os.listdir(audio_path):
    actor_path = os.path.join(audio_path, actor_folder)
    for file_name in os.listdir(actor_path):
        file_path = os.path.join(actor_path, file_name)
        parts = file_name.split("-")
        if len(parts) < 3:
            continue
        emotion = emotions[parts[2].split(".")[0]]  # Get emotion from file name
        if emotion not in observed_emotions:
            continue
        features = extract_features(file_path)
        data.append(features)
        labels.append(emotion)

X = np.array(data)
y = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


In [21]:
print(X_train.shape[0],y_train.shape[0])

2040 2040


## Model Training

In [22]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08,hidden_layer_sizes=(300,),learning_rate='adaptive',max_iter=500)
model.fit(X_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

## Model Evaluation

In [23]:
y_pred=model.predict(X_test)


## calculate the accuracy of the model

In [24]:
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.30%


In [25]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.85      0.76      0.80        72
        calm       0.67      0.71      0.69        68
     disgust       0.80      0.50      0.62        64
     fearful       0.70      0.85      0.77        72
       happy       0.68      0.66      0.67        64
     neutral       0.43      0.57      0.49        35
         sad       0.72      0.52      0.60        73
   surprised       0.62      0.84      0.72        63

    accuracy                           0.68       511
   macro avg       0.68      0.68      0.67       511
weighted avg       0.70      0.68      0.68       511

